<a href="https://colab.research.google.com/github/AnastasiaZAYU/Data_Analysis_2023/blob/main/Lab3_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Виконати наступне:
1. **Повнозв'язані нейронні мережі.**
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше.

2. **Згорткові нейронні мережі.**
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

3. **Рекурентні нейронні мережі.**
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі. Результати порівняйте з одержаними раніш.

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Повнозв'язані нейронні мережі

### Завантаження та очишення даних

In [73]:
df=pd.read_csv('advertising_ef.csv')
df=df.dropna()

### Вивід назв колонок

In [74]:
df.columns

Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Gender', 'Clicked on Ad'],
      dtype='object')

### Відокремлення target та стандартизація даних

In [75]:
X = df.drop(['Clicked on Ad'], axis = 1)
y = np.array(df['Clicked on Ad'])
print(X.shape)

(980, 5)


In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.25, random_state=10)

In [78]:
scaler = MinMaxScaler()
scaler.fit(X_train)
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)

In [84]:
model = keras.Sequential([
 layers.Dense(32, activation="relu", input_shape=(X_train.shape[-1],)),
 layers.Dropout(0.3),
 layers.Dense(16, activation="relu"),
 layers.Dropout(0.3),
 layers.Dense(1, activation="sigmoid")
])

In [85]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 32)                192       
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dropout_9 (Dropout)         (None, 16)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 737 (2.88 KB)
Trainable params: 737 (2.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [90]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512)

Epoch 1/50
2/2 [==============================] - 0s 331ms/step - loss: 0.0875 - accuracy: 0.9755 - val_loss: 0.1228 - val_accuracy: 0.9633
Epoch 2/50
2/2 [==============================] - 0s 112ms/step - loss: 0.0779 - accuracy: 0.9796 - val_loss: 0.1231 - val_accuracy: 0.9633
Epoch 3/50
2/2 [==============================] - 0s 157ms/step - loss: 0.0873 - accuracy: 0.9728 - val_loss: 0.1229 - val_accuracy: 0.9633
Epoch 4/50
2/2 [==============================] - 0s 114ms/step - loss: 0.0825 - accuracy: 0.9741 - val_loss: 0.1223 - val_accuracy: 0.9633
Epoch 5/50
2/2 [==============================] - 0s 111ms/step - loss: 0.0824 - accuracy: 0.9769 - val_loss: 0.1233 - val_accuracy: 0.9633
Epoch 6/50
2/2 [==============================] - 0s 142ms/step - loss: 0.0865 - accuracy: 0.9782 - val_loss: 0.1235 - val_accuracy: 0.9633
Epoch 7/50
2/2 [==============================] - 0s 143ms/step - loss: 0.0947 - accuracy: 0.9714 - val_loss: 0.1232 - val_accuracy: 0.9633
Epoch 8/50
2/2 [====

In [93]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.12304538488388062
Test accuracy: 0.9632652997970581


## 2. Згорткові нейронні мережі

In [100]:
import zipfile
import os
import shutil

# Шлях до вашого архіву з датасетом
zip_path = '/content/archive.zip'

# Розпакувати архів
extracted_path = '/content/extracted_data'  # шлях для розпакованих даних
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Створити теки для train і test
train_path = '/content/train'
test_path = '/content/test'
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Шляхи до тек Test і Train
source_train_path = os.path.join(extracted_path, 'PandasBears', 'Train')
source_test_path = os.path.join(extracted_path, 'PandasBears', 'Test')

# Функція для копіювання файлів з однієї теки в іншу
def copy_files(source, destination):
    for root, dirs, files in os.walk(source):
        for file in files:
            # Отримати шлях до кожного файлу
            file_path = os.path.join(root, file)
            # Отримати шлях для файлу у папці призначення
            relative_path = os.path.relpath(file_path, source)
            destination_file_path = os.path.join(destination, relative_path)
            # Створити теку, якщо її не існує
            os.makedirs(os.path.dirname(destination_file_path), exist_ok=True)
            # Копіювати файл
            shutil.copy(file_path, destination_file_path)

# Копіювати файли з теки 'Train/Pandas' у теку 'train/Pandas'
copy_files(os.path.join(source_train_path, 'Pandas'), os.path.join(train_path, 'Pandas'))

# Копіювати файли з теки 'Train/Bears' у теку 'train/Bears'
copy_files(os.path.join(source_train_path, 'Bears'), os.path.join(train_path, 'Bears'))

# Копіювати файли з теки 'Test/Pandas' у теку 'test/Pandas'
copy_files(os.path.join(source_test_path, 'Pandas'), os.path.join(test_path, 'Pandas'))

# Копіювати файли з теки 'Test/Bears' у теку 'test/Bears'
copy_files(os.path.join(source_test_path, 'Bears'), os.path.join(test_path, 'Bears'))

In [108]:
batch_size = 32
image_height = 180
image_width = 180

In [109]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale=1./255)  # Нормалізація пікселів для тренувального датасету
test_data_generator = ImageDataGenerator(rescale=1./255)  # Нормалізація пікселів для тестового датасету

# Шляхи до тек з даними
train_data_dir = '/content/train'
test_data_dir = '/content/test'

# Зчитування та підготовка тренувальних та тестових даних
train_generator = train_data_generator.flow_from_directory(
        train_data_dir,
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode='binary')  # Якщо у вас є два класи, використайте 'binary'; для більшої кількості класів - 'categorical'

test_generator = test_data_generator.flow_from_directory(
        test_data_dir,
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode='binary')  # Те ж саме, що й для тренувального датасету


Found 500 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Зменшення перенавчання
    Dense(1, activation='sigmoid')  # Вихідний шар для бінарної класифікації
    # Якщо класів більше ніж два, використовуйте Dense зі відповідною кількістю нейронів та softmax активацією
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [111]:
num_epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)


Epoch 1/10
15/15 [==============================] - 35s 2s/step - loss: 0.4851 - accuracy: 0.7991 - val_loss: 0.0229 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 32s 2s/step - loss: 0.0385 - accuracy: 0.9872 - val_loss: 3.2859e-05 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 34s 2s/step - loss: 0.0473 - accuracy: 0.9829 - val_loss: 3.9322e-04 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 32s 2s/step - loss: 0.0241 - accuracy: 0.9936 - val_loss: 4.4502e-04 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 36s 2s/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 5.4784e-04 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 32s 2s/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 8.1791e-05 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 32s 2s/step - loss: 7.5753e-04 - accuracy: 1.0000 - val_loss: 2.4901e-05 - val_accuracy:

In [112]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_accuracy}")


3/3 [==============================] - 2s 498ms/step - loss: 1.0991e-05 - accuracy: 1.0000
Test accuracy: 1.0


## 3. Рекурентні нейронні мережі

### Завантаження та очищення датасету
https://www.kaggle.com/datasets/joebeachcapital/restaurant-reviews

In [150]:
df=pd.read_csv('Restaurant_reviews.csv')
df=df.dropna()

### Вивід назв колонок та видалення зайвих

In [151]:
df.columns

Index(['Restaurant', 'Reviewer', 'Review', 'Rating'], dtype='object')

In [152]:
df=df.drop(['Restaurant','Reviewer'],axis=1)

### Підрахунок кількості елементів кожного класа

In [153]:
from collections import Counter
Counter( df['Rating'])

Counter({5: 3826, 4: 2373, 1: 1735, 3: 1192, 2: 684})

### Зменшення кількості класів, шляхом їх об'єднання

In [154]:
df['Rating'].replace({2: 1, 4: 3},inplace=True)
Counter( df['Rating'])

Counter({5: 3826, 3: 3565, 1: 2419})

### Очищення тексту регулярними виразами

In [155]:
def clean_text(text):
  stop_words = set(stopwords.words("english"))
  stop_words.update(['restaurant','place','food','taste','service','ordered','order','one','good'])
  text = text.lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  text = re.sub('\n\S+', '', text)
  text = re.sub('\s{2,}', " ", text)
  text = ' '.join([word for word in text.split(' ') if word not in stop_words])
  return text

In [156]:
df['Review']=df['Review'].apply(clean_text)

In [157]:
df['Review']

0       ambience quite saturday lunch cost effective s...
1       ambience pleasant evening prompt experience so...
2       must try great great ambience thnx pradeep sub...
3       soumen das arun great guy behavior sincerety c...
4       goodwe kodi drumsticks basket mutton biryani t...
                              ...                        
9805    never fan chinese visited chinese pavilionbanj...
9806    visited friends immediately blown away quality...
9807    im going cut chase excellent must say honey ch...
9808    madhumathi mahajan well start nice courteous s...
9809    personally love prefer chinese couple times hu...
Name: Review, Length: 9810, dtype: object

### Розбиття датасету на тренувальні і тестові дані

In [158]:
X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Rating'], test_size=0.30, random_state=23)

In [138]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
maxlen = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index
print(f'Знайдено {len(word_index)} унікальних токенів.')

data = pad_sequences(X_train_seq, maxlen=maxlen)

Знайдено 14260 унікальних токенів.


In [141]:
max_length = 100
padding_type = "post"
trunction_type="post"
X_train_pad = keras.utils.pad_sequences(X_train_seq,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
X_test_pad = keras.utils.pad_sequences(X_test_seq,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

In [142]:
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.models import Sequential

embedding_dim = 100  # Розмірність векторного представлення слів
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=maxlen),
    SpatialDropout1D(0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_train))